## Keras - Save and Restore Models

#### Preparation

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

1.11.0


In [2]:
# Get data: first 1000 images from MNIST
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

In [3]:
print("Before reshaping: ", test_images.shape)

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print("After reshaping: ", test_images.shape)

Before reshaping:  (10000, 28, 28)
After reshaping:  (1000, 784)


#### Build the model

In [4]:
# Return a short sequential model

def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    
    return model

# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### Save checkpoints during training

The primary use case is to automatically save checkpoints *during* and at *the end* of training. This way you can use a trained model without having to retrain it, or picking-up training where you left of -- in case the training process was interrupted.

The callback function designed for this purpose is `tf.keras.callbacks.ModelCheckpoint`. This callback takes a couple of arguments to configure checkpointing.

In [15]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2281 - acc: 0.6382
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 679us/step - loss: 1.1400 - acc: 0.6640 - val_loss: 0.7460 - val_acc: 0.7750
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4198 - acc: 0.8789
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 284us/step - loss: 0.4221 - acc: 0.8780 - val_loss: 0.5392 - val_acc: 0.8260
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2894 - acc: 0.9203
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 319us/step - loss: 0.2910 - acc: 0.9230 - val_loss: 0.4702 - val_acc: 0.8500
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.1992 - acc: 0.9594
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000 [==

In [16]:
!ls {checkpiont_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


#### Load the stored checkpoint

In [17]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 236us/step
Untrained model, accuracy:  9.90%


In [18]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 85us/step
Untrained model, accuracy: 86.80%


#### Configuring callback options

In [19]:
# Include the epoch in the file name
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Save weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True, period=5)

model = create_model()
model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback],
         validation_data=(test_images, test_labels), verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [20]:
!ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001  cp-0050.ckpt.index
cp-0025.ckpt.index


In [22]:
last = tf.train.latest_checkpoint(checkpoint_dir)
last

'training_2/cp-0050.ckpt'

In [24]:
# Reset model and load the latest checkpoint
model = create_model()
model.load_weights(last)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 176us/step
Restored model, accuracy: 87.10%


#### Manually save weights

In [25]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 183us/step
Restored model, accuracy: 87.10%


#### Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later -- from the exact same state -- without access to the original code.

You can save a Keras model, load them in Tensorflow.js and then train and run them in web browsers!

In [26]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5') # HDF5 file

Epoch 1/5
1000/1000 [==============================] - 1s 516us/step - loss: 1.1988 - acc: 0.6640
Epoch 2/5
1000/1000 [==============================] - 0s 215us/step - loss: 0.4211 - acc: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 234us/step - loss: 0.2919 - acc: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 268us/step - loss: 0.2095 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 280us/step - loss: 0.1606 - acc: 0.9640


In [27]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

new_model.compile(optimizer=tf.train.AdamOptimizer(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 317us/step
Restored model, accuracy: 85.50%
